In [26]:
import pickle 
import json
from tqdm import tqdm 

In [73]:
def load_data(filename):
    with open(f"{filename}.pkl", "rb") as f:
        data = pickle.load(f)
    print("Loaded data!")
    return data
        
def save_data(data, filename):
    with open(f"{filename}.pkl", "wb") as f:
        pickle.dump(data, f)
        
def export_to_json(data, filename):
    with open(f'{filename}.json', 'w') as json_file:
      json.dump(data, json_file)

In [50]:
data = load_data("player_matches")
match_ids = list(data.keys())

Loaded data!


In [30]:
def download_team_players():
    from espncricinfo.match import Match
    count = 0
    for m_id in match_ids:
        count += 1
        print(count)
        if 'team_1_players' not in data[m_id].keys():
            m = Match(m_id)
            data[m_id]['team_1_players'] = m.team_1_players
            data[m_id]['team_2_players'] = m.team_2_players
            if count % 100 == 0:
                print(count)
                save_data(data, "player_matches")

In [51]:
player_dict = {}
for idx in tqdm(match_ids): 
    m = data[idx]
    temp = []
    for player in m['team_1_players']:
        p_id = player['player_id']
        if p_id not in player_dict:
            player_dict[p_id] = player
        temp.append(p_id)
    data[idx]['team_1_players'] = temp
    
    temp=[]
    for player in m['team_2_players']:
        p_id = player['player_id']
        if p_id not in player_dict:
            player_dict[p_id] = player
        temp.append(p_id)
    data[idx]['team_2_players'] = temp






100%|██████████| 1920/1920 [00:00<00:00, 20032.30it/s]


In [52]:
len(player_dict.keys())

1391

In [83]:
player_ids = data[match_ids[340]]['team_1_players']

unwanted_data = ['age_days', 'age_years', 'captain', 'keeper', 'object_id', 'popular_name', 'portrait_alt_id', 'portrait_object_id', 'status_id']
for p_id, player in player_dict.items():
    for col in unwanted_data:
        try:
            del player[col]
        except KeyError:
            continue

In [84]:
for k, v in player_dict['1934'].items():
    print(f"{k.upper():<20}: {v:<10}")

ALPHA_NAME          : TENDULKAR,SR
BATTING_HAND        : right-hand batsman
BATTING_STYLE       : rhb       
BATTING_STYLE_LONG  : right-hand bat
BOWLING_HAND        : right-arm bowler
BOWLING_PACESPIN    : spin bowler
BOWLING_STYLE       : ob/lbg    
BOWLING_STYLE_LONG  : right-arm offbreak /legbreak googly 
CARD_LONG           : SR Tendulkar
CARD_QUALIFIER      :           
CARD_SHORT          : Tendulkar 
DOB                 : 1973-04-24
KNOWN_AS            : Sachin Tendulkar
MOBILE_NAME         : Tendulkar 
PLAYER_ID           : 1934      
PLAYER_PRIMARY_ROLE : top-order batsman
PLAYER_STYLE_ID     : 2         
PLAYER_TYPE         : 1         
PLAYER_TYPE_NAME    : player    


In [109]:
save_data(player_dict, "player_table")

In [110]:
export_to_json(player_dict, 'player_table')

In [108]:
def checkType(players):
    for p in players:
        if not isinstance(p, str): return False
        
    if len(players) == 11:
        return True

count = 0
for i, m in data.items():
    if not checkType(m['team_1_players']) or not checkType(m['team_2_players']):
        count += 1
        
print(f"{count} matches with 12 players.")

73 matches with 12 players.


In [112]:
save_data(data, "matches")

In [113]:
export_to_json(data, "matches")

In [128]:
from collections import defaultdict
def getPlayerByName(name, player_dict):
    possibilities = []
    for i, profile in player_dict.items():
        if name in profile['known_as']:
            possibilities.append(profile)
    return possibilities
    
name = getPlayerByName('Mark Wood', player_dict)
print(name)
name = name[0]['player_id']

count = 0
for i, m in data.items():
    if name in m['team_1_players'] or name in m['team_2_players']:
        count += 1
print(count)

[{'alpha_name': 'WOOD,MA', 'batting_hand': 'right-hand batsman', 'batting_style': 'rhb', 'batting_style_long': 'right-hand bat', 'bowling_hand': 'right-arm bowler', 'bowling_pacespin': 'pace bowler', 'bowling_style': 'rf', 'bowling_style_long': 'right-arm fast ', 'card_long': 'MA Wood', 'card_qualifier': '', 'card_short': 'Wood', 'dob': '1990-01-11', 'known_as': 'Mark Wood', 'mobile_name': 'Wood', 'player_id': '56993', 'player_primary_role': 'bowler', 'player_style_id': '4', 'player_type': '1', 'player_type_name': 'player'}]
51
